# 1. '라면,통조림,상옥즉석' 중분류에 대한 장바구니 분석
- 이거는 일단 해본거

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 장바구니 분석을 수행하는 함수
def basket_analysis(data):
    unique_dates = data['판매일'].unique()
    final_results = pd.DataFrame()

    for sale_date in unique_dates:
        # 판매일 별로 데이터 필터링
        date_data = data[data['판매일'] == sale_date]

        # 대분류가 '면류.라면류'인 항목만 필터링
        date_data = date_data[date_data['대분류'] == '면류.라면류']

        # 매출처코드별로 대분류를 one-hot encoding하여 장바구니 행렬 생성
        basket = (date_data
                  .groupby(['매출처코드', '대분류'])['판매일']
                  .count()
                  .unstack()
                  .reset_index()
                  .fillna(0)
                  .set_index('매출처코드'))

        # 1과 0으로 이진화
        basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

        # Apriori 알고리즘을 사용하여 빈번한 아이템 집합 찾기 (지지도 최소값 0.01)
        frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

        # 연관 규칙 생성
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

        # Antecedents에 '라면, 통조림, 상온즉석'이 포함된 규칙만 필터링
        rules_filtered = rules[rules['antecedents'].apply(lambda x: '면류.라면류' in x)]

        # 판매일자를 추가하여 결과 저장
        if not rules_filtered.empty:
            rules_filtered = rules_filtered.copy()
            rules_filtered['판매일'] = sale_date  # 판매일 추가
            final_results = pd.concat([final_results, rules_filtered], axis=0)

    # '판매일' 컬럼을 맨 앞으로 이동
    if '판매일' in final_results.columns:
        final_results = final_results[['판매일'] + [col for col in final_results.columns if col != '판매일']]

    return final_results

In [3]:
# 실제 실행 코드
# 데이터를 업로드하시면 해당 데이터를 data 변수로 활용하세요.
data = pd.read_excel('(2데이터)옵션코드별 이상치 처리 병합_주 단위(3).xlsx', engine='openpyxl')  # 업로드한 파일 경로 사용

results = basket_analysis(data)
save_results(results, '면류,라면류 장바구니 분석 결과.csv')
print("CSV 파일이 저장되었습니다.")

FileNotFoundError: [Errno 2] No such file or directory: '(2데이터)옵션코드별 이상치 처리 병합_주 단위(3).xlsx'

# 2. 전체 중분류 장바구니 분석 결과

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 장바구니 분석을 수행하는 함수
def basket_analysis(data):
    unique_dates = data['판매일'].unique()
    final_results = pd.DataFrame()

    for sale_date in unique_dates:
        # 판매일 별로 데이터 필터링
        date_data = data[data['판매일'] == sale_date]
        
        # 매출처코드별로 중분류를 one-hot encoding하여 장바구니 행렬 생성
        basket = (date_data
                  .groupby(['매출처코드', '대분류'])['판매일']
                  .count()
                  .unstack()
                  .reset_index()
                  .fillna(0)
                  .set_index('매출처코드'))

        # 1과 0으로 이진화
        basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

        # Apriori 알고리즘을 사용하여 빈번한 아이템 집합 찾기 (지지도 최소값 0.01)
        frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

        # 연관 규칙 생성
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

        # 판매일자를 추가하여 결과 저장
        if not rules.empty:
            rules = rules.copy()
            rules['판매일'] = sale_date  # 판매일 추가
            final_results = pd.concat([final_results, rules], axis=0)

    # '판매일' 컬럼을 맨 앞으로 이동
    if '판매일' in final_results.columns:
        final_results = final_results[['판매일'] + [col for col in final_results.columns if col != '판매일']]

    return final_results

# CSV로 결과 저장
def save_results(results, output_file):
    results.to_csv(output_file, index=False)

In [ ]:
# 실제 실행 코드
data = pd.read_excel('(2데이터)옵션코드별 이상치 처리 병합_주 단위.xlsx')  # 업로드한 파일 경로 사용

results = basket_analysis(data)
save_results(results, '전체 대분류 장바구니 분석 결과.csv')
print("CSV 파일이 저장되었습니다.")

## 2.1 판매일자별 antecedents의 중분류1개, consequents의 중분류1개 인 행만 남기기

In [ ]:
total = pd.read_csv('전체 대분류 장바구니 분석 결과.csv',encoding='utf-8')
total

In [ ]:
# antecedents와 consequents 각각의 길이가 1인 경우만 남기기
filtered_total = total[total['antecedents'].apply(lambda x: len(eval(x)) == 1) & total['consequents'].apply(lambda x: len(eval(x)) == 1)]
filtered_total

In [ ]:
filtered_total = filtered_total.reset_index(drop=True)
filtered_total

In [ ]:
# 20개
filtered_total['antecedents'].unique()

In [ ]:
# 15개
filtered_total['consequents'].unique()

In [ ]:
filtered_total[filtered_total['판매일']=='2021-01-04']

In [ ]:
# frozenset을 없애고 안의 값만 추출하는 함수
def extract_values(frozenset_str):
    # eval()로 frozenset을 리스트로 변환한 후, 첫 번째 값을 문자열로 변환
    return ', '.join(list(eval(frozenset_str)))

# antecedents와 consequents에서 frozenset과 '' 제거
filtered_total['antecedents'] = filtered_total['antecedents'].apply(extract_values)
filtered_total['consequents'] = filtered_total['consequents'].apply(extract_values)

# # 원하는 컬럼만 남기기
# filtered_total = filtered_total[['판매일', 'antecedents', 'consequents', 'confidence']]
# filtered_total

In [ ]:
filtered_total

In [ ]:
# CSV 파일로 저장 (필요시)
filtered_total.to_csv('1대1 대응 장바구니 분석 결과.csv', index=False)

# 데이터 병합

In [ ]:
week = pd.read_excel('(2데이터)옵션코드별 이상치 처리 병합_주 단위.xlsx')  # 업로드한 파일 경로 사용
week

In [ ]:
week['대분류'].unique()

In [ ]:
week['중분류'].nunique()

In [ ]:
# '중분류' 값을 기준으로 더미 컬럼 생성
week_dummies = pd.get_dummies(week['대분류'])

# 기존 week 데이터프레임에 더미 컬럼 추가
week_with_dummies = pd.concat([week, week_dummies], axis=1)

In [ ]:
week_with_dummies

In [ ]:
# '1대1 대응 장바구니 분석 결과.csv'와 'week_with_dummies' 파일 불러오기
basket_analysis_result = pd.read_csv('1대1 대응 장바구니 분석 결과.csv')

# 각 행에 대해 조건을 만족하는 경우 confidence 값을 채워넣기
for _, row in basket_analysis_result.iterrows():
    # 판매일, antecedents, consequents, confidence 값 추출
    sale_date = row['판매일']
    antecedent = row['antecedents']  # 이미 문자열이므로 그대로 사용
    consequent = row['consequents']  # 이미 문자열이므로 그대로 사용
    confidence_value = row['confidence']
    
    # 해당 조건을 만족하는 행 찾기 (판매일과 중분류가 일치하는 경우)
    condition = (week_with_dummies['판매일'] == sale_date) & (week_with_dummies['대분류'] == antecedent)
    
    # 해당하는 셀에 confidence 값을 삽입
    week_with_dummies.loc[condition, consequent] = confidence_value

# 이제 남아있는 값들에 대해 False -> 0, True -> 1 변환
# 중분류 값에서 파생된 22개의 열들만 변환
subcategories_columns = week_with_dummies.columns[week_with_dummies.columns.isin(basket_analysis_result['consequents'])]

# 해당 컬럼들에 대해 남은 값들을 변환 (False -> 0, True -> 1)
week_with_dummies[subcategories_columns] = week_with_dummies[subcategories_columns].applymap(lambda x: 1 if x == True else (0 if x == False else x))

In [ ]:
week_with_dummies

In [ ]:
# 결과 확인 후 저장
week_with_dummies.to_excel('주차별 장바구니 분석 결과 병합.xlsx', index=False)

# 3. 공공데이터 병합

In [ ]:
ggdata = pd.read_excel('공공데이터.xlsx')
ggdata

In [ ]:
ggdata['연월'] = ggdata['연월'].astype('str')
ggdata.info()

In [ ]:
ggdata

In [ ]:
# 특정 연도의 '1'을 '10'으로 변경하는 코드
def custom_change(x):
    year, month = x.split('.')
    if month == '1' and year in ['2021', '2022', '2023']:
        return f'{year}.10'
    else:
        return f'{year}.{month.zfill(2)}'

# 연월 컬럼에 적용
ggdata['연월'] = ggdata['연월'].apply(custom_change)

In [ ]:
ggdata

In [ ]:
# 장바구니 결과 불러오기
import pandas as pd
com = pd.read_excel('주차별 장바구니 분석 결과 병합.xlsx')
com

In [ ]:
# 연월 컬럼 만들기
com['연월'] = com['판매일'].dt.strftime('%Y.%m')
com

In [ ]:
# 조인
merged_data = pd.merge(com, ggdata, on='연월', how='left')
merged_data

# 4. 평균 발주일자 병합

In [ ]:
ord = pd.read_excel('매출처코드별 평균발주일자 및 총발주건수.xlsx')
ord

In [ ]:
merged_data_total = pd.merge(merged_data, ord, on='매출처코드', how='left')
merged_data_total

# 5. 중분류가 '라면,통조림,상온즉석'인 행만 추출

In [ ]:
data1 = merged_data_total[merged_data_total['대분류']=='면류.라면류'].reset_index(drop=True)
data1

In [ ]:
# 연월 컬럼 제거
data1 = data1.drop('연월', axis=1)
data1

In [ ]:
data1.to_csv('데이터2 라면,통조림,상온즉석 통합.csv')